# [全局解释器锁（GIL： Global Interpreter Lock）](https://www.cnblogs.com/linhaifeng/articles/7449853.html "GIL")

- 保证python解释器同一时间只能执行一个任务的代码 </br>
![这是图片](../../../image\00\GIL执行流程1.png "保证python解释器同一时间只能执行一个任务的代码")

- GIL保护的是解释器级的数据，保护用户自己的数据则需要自己加锁处理，如下图 </br>
![这是图片](../../../image\00\GIL执行流程2.png "GIL保护的是解释器级的数据，保护用户自己的数据则需要自己加锁处理，如下图")

- GIL与多线程

有了GIL的存在，同一时刻同一进程中只有一个线程被执行

听到这里，有的同学立马质问：进程可以利用多核，但是开销大，而python的多线程开销小，但却无法利用多核优势，也就是说python没用了，php才是最牛逼的语言？

别着急啊，老娘还没讲完呢。

要解决这个问题，我们需要在几个点上达成一致：

1. cpu到底是用来做计算的，还是用来做I/O的？

2. 多cpu，意味着可以有多个核并行完成计算，所以多核提升的是计算性能

3. 每个cpu一旦遇到I/O阻塞，仍然需要等待，所以多核对I/O操作没什么用处 
一个工人相当于cpu，此时计算相当于工人在干活，I/O阻塞相当于为工人干活提供所需原材料的过程，工人干活的过程中如果没有原材料了，则工人干活的过程需要停止，直到等待原材料的到来。

如果你的工厂干的大多数任务都要有准备原材料的过程（I/O密集型），那么你有再多的工人，意义也不大，还不如一个人，在等材料的过程中让工人去干别的活，

反过来讲，如果你的工厂原材料都齐全，那当然是工人越多，效率越高

 
结论：

1. 对计算来说，cpu越多越好，但是对于I/O来说，再多的cpu也没用

2. 当然对运行一个程序来说，随着cpu的增多执行效率肯定会有所提高（不管提高幅度多大，总会有所提高），这是因为一个程序基本上不会是纯计算或者纯I/O，所以我们只能相对的去看一个程序到底是计算密集型还是I/O密集型，从而进一步分析python的多线程到底有无用武之地

#分析：
我们有四个任务需要处理，处理方式肯定是要玩出并发的效果，解决方案可以是：
方案一：开启四个进程
方案二：一个进程下，开启四个线程

#单核情况下，分析结果: 
　　如果四个任务是计算密集型，没有多核来并行计算，方案一徒增了创建进程的开销，方案二胜
　　如果四个任务是I/O密集型，方案一创建进程的开销大，且进程的切换速度远不如线程，方案二胜

#多核情况下，分析结果：
　　如果四个任务是计算密集型，多核意味着并行计算，在python中一个进程中同一时刻只有一个线程执行用不上多核，方案一胜
　　如果四个任务是I/O密集型，再多的核也解决不了I/O问题，方案二胜

#结论：现在的计算机基本上都是多核，python对于计算密集型的任务开多线程的效率并不能带来多大性能上的提升，甚至不如串行(没有大量切换)，但是，对于IO密集型的任务效率还是有显著提升的。

In [3]:
%%writefile test_thread.py
# 计算密集型：多进程效率高
from multiprocessing import Process
from threading import Thread
import os ,time 

def work():
    res=0
    for i in range(100000000):
        res*=i
        
if __name__ == "__main__":
    print(os.cpu_count()) #* 12核
    l=[]
    start_time = time.time()
    for _ in range(12):
        # p=Process(target=work) #* run time is 13.6089768409729
        # l.append(p)
        # p.start()
        t=Thread(target=work) #* run time is 50.40143013000488
        l.append(t)
        t.start()
        
    for p in l:
        p.join()
    
    end_time=time.time()
    print(f'run time is {end_time-start_time}' ) 

Overwriting test_thread.py


In [6]:
%%writefile test_thread.py
# I/O密集型：多线程效率高
from multiprocessing import Process
from threading import Thread
import os,time
def work():
    time.sleep(2)

if __name__ == '__main__':
    l=[]
    print(os.cpu_count()) #本机为12核
    start=time.time()
    for _ in range(400):
        # p=Process(target=work) #耗时12s多,大部分时间耗费在创建进程上 run time is 8.708985090255737
        p=Thread(target=work) # 耗时2s多 run time is 2.059626340866089
        l.append(p)
        p.start()
    for p in l:
        p.join()
    stop=time.time()
    print(f'run time is {stop-start}' ) 

Overwriting test_thread.py
